<a href="https://colab.research.google.com/github/Daryldactyl/Kaggle_Data/blob/main/insurance_prices_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import required libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
insurance = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [ ]:
insurance.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [ ]:
# Preprocessing with a column transformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

## Evaluation trial for Normlization

* I experimented with both MinMax Scaler and Standard Scaler and found I got the best accuracy on the Standard Scaler

In [ ]:
#Create the column transformer
ct = make_column_transformer(
    (StandardScaler(), ['age', 'bmi', 'children']),
    (OneHotEncoder(handle_unknown='ignore'), ['sex', 'smoker', 'region'])
)

#Create X and y
X = insurance.drop('charges', axis=1)
y = insurance['charges']

#Build our train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fit the column transformer to the training data
ct.fit(X_train)

#Transform the training and testing data with normalization and OneHotEncoder
X_train_trans = ct.transform(X_train)
X_test_trans = ct.transform(X_test)

## Evaluation trial Model Building

The model began as a Dense input of 10 units and an output of 1

###Evolution:
* Attempt 1: Added another Dense layer of 100 and tested.
* Attempt 2: Changed the Dense layer to 50 and found better results
* Attempt 3: Added another Dense Layer of 50 and saw improvement
* Attempt 4: Changed my 10 layer to 100
* Attempt 5: Added a Dense of 50 again after my 100 unit layer
* Attempt 6: After extensive research on activation options happened across the Leaky Relu layer. Changed this to the input and model improved drastically

###Evolution of the Compiler
 * Attempt 1: Had the Optimizer as SGD with base learning rate
 * Attempt 2: Changed Optimizer to Adam with base learning rate
 * Attempt 3: Changed Learning Rate to 0.01 and saw the best results

In [ ]:
ins_model = tf.keras.Sequential([
    tf.keras.layers.LeakyReLU(input_shape=[11]),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

ins_model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.01), loss = 'mae', metrics=['mae'])

ins_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 leaky_re_lu (LeakyReLU)     (None, 11)                0         
                                                                 
 dense (Dense)               (None, 50)                600       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 100)               5100      
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 13351 (52.15 KB)
Trainable params: 13351 (

#Evolution of Fit Method

* Attempt 1: Ran 10 epochs with no good convergence
* Attempt 2: Ran with 100 epochs and got better results
* Attempt 3: Ran with 250 epochs and results improved
* Attempt 4: Changed batch size to 10 and results were much better with convergence
* Attempt 5: Upped to 500 epochs with batch size 10 (probably could have included early stoppage measures to reduce training time)

In [ ]:
history = ins_model.fit(X_train_trans, y_train, epochs=500, verbose=0, batch_size=10)

In [ ]:
ins_model.evaluate(X_test_trans, y_test)

9/9 [==============================] - 0s 3ms/step - loss: 1345.1526 - mae: 1345.1526


[1345.152587890625, 1345.152587890625]

##Showing comparisons of Predictions

In [ ]:
y_pred = ins_model.predict(X_test_trans)

9/9 [==============================] - 0s 3ms/step


In [ ]:
# Convert y_pred to a Series and align the indices with y_test
y_pred_series = pd.Series(tf.squeeze(y_pred), name='Predicted', index=y_test.index)

# Create a DataFrame with both predicted and actual values
predictions_df = pd.concat([y_pred_series, y_test.rename('Actual')], axis=1)

# Drop rows with NaN values
predictions_df = predictions_df.dropna()

# Print the DataFrame
print(predictions_df)

         Predicted       Actual
764    8992.158203   9095.06825
887    5143.559570   5272.17580
890   28710.597656  29330.98315
1293   9304.535156   9301.89355
259   33478.726562  33750.29180
...            ...          ...
109   46862.210938  47055.53210
575   12092.988281  12222.89830
535    5914.522461   6067.12675
543   50644.480469  63770.42801
846    9742.302734   9872.70100

[268 rows x 2 columns]
